In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [2]:
doc_A = pd.read_excel("DocA.xlsx", header = 0)
doc_B = pd.read_excel("DocB.xlsx", header = 0)

In [3]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [4]:
doc_A['WORK ORDER Narration'] = doc_A['WORK ORDER Narration'].apply(clean_text)
doc_B['WORK ORDER Narration'] = doc_B['WORK ORDER Narration'].apply(clean_text)

In [5]:
docA_desc = doc_A['WORK ORDER Narration'].tolist()
docB_desc = doc_B['WORK ORDER Narration'].tolist()

In [6]:
def Jaccard_Similarity(doc1, doc2): 
    
    # List the unique words in a document
    words_doc1 = set(doc1.lower().split(' ')) 
    words_doc2 = set(doc2.lower().split(' '))
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [7]:
high_match_wo = []
high_match_score = []

In [8]:
docA_WO = doc_A['WORK ORDER'].tolist()

In [9]:
for i in docB_desc:
    match_scores = []
    for j in docA_desc:
        sim = Jaccard_Similarity(i,j)
        match_scores.append(sim)
    maxpos = match_scores.index(max(match_scores))
    high_match_wo.append(docA_WO[maxpos])
    high_match_score.append(max(match_scores))
    

In [10]:
len(high_match_wo), len(docB_desc)

(293, 293)

In [11]:
output_df = pd.DataFrame({
    "Work Order":doc_B['WORK ORDER'].tolist(),
    "Matched Work Order":high_match_wo
})

In [12]:
output_df.to_excel("Matched Work Orders.xlsx", index = False)